# Execution Graph (DAG) & Programming Interface

This is the most important tutorial you need to read through.

The construction of workflow and the construction of ExecutionEngines can be totally decoupled. But you can also couple them, this may be useful when you need to use certain config in an ExecutionEngine. 

In fugue, workflow is static, it is strictly the description or **spec** of your logic flow, and it has nothing to do with execution. When you finish writing the workflow spec, you can also choose a Fugue ExecutionEngine and an Adagio workflow ExecutionEngine for the end to end execution. The Adagio workflow ExecutionEngine will turn your workflow _spec_ into a workflow _instance_ (nodes specs will also turn to node instances) and execute in certain order. For example Adagio default ExecutionEngine `SequentialExecutionEngine` will run everything sequentially in the order you list in the workflow spec. In each node of the execution graph, it will use the given Fugue ExecutionEngine. And the edges of the graph is always `DataFrame`


## Graph Nodes - Extensions

There are only 3 types of nodes in Fugue workflow. They are also called driver side extensions.

<img src="../images/nodes.svg" width="500">

* **Creator**: no input, single output dataframe, it is to produce dataframe input for other types of nodes, for example load file or create mock data
* **Processor**: one or multiple input dataframes, single output dataframe, it is to do certain transformation and pass to the next node
* **Outputter**: one or multiple input dataframes, no input, it is to finalize the process of the input, for example save or print

All these nodes/extensions are executed on driver side and they are ExecutionEngine aware. For example if you really want to use certain features of RDD, you can write your native Spark code in a `Processor` because inside a processor, you can access the ExecutionEngine and if it is SparkExecutionEngine, you can get `spark_session`. There will be examples in this tutorial.

There are two special types of `Processor`: `Transformer` and `CoTransformer`. They are special because they are NOT ExeuctionEngine aware, and their exposed interface is to describe the job to do on worker side not driver side.

<img src="../images/transformers.svg" width="300">

* **Transformer**: single `LocalDataFrame` in, single `LocalDataFrame` out
* **CoTransformer**: one or multiple `LocalDataFrame` in, single `LocaDataFrame` out

They only care about how to process a partition of the dataframe(s) on a local machine.

| . | Creator | Processor | Outputter | Transformer | CoTransformer
|---|---|---|---|---|---
|Input | 0    | 1+        | 1+        | 1           | 1+
|Output| 1    | 1         | 0         | 1           | 1
|Side  |Driver|Driver     | Driver    | Worker      | Worker
|Engine Aware | Yes | Yes | Yes       | No          | No

A Fugue Workflow consists of these extensions, we orchestrate the extensions.


## Initialize a Workflow

In [ ]:
from fugue.workflow import FugueWorkflow

dag = FugueWorkflow()
df = dag.df([[0]],"a:int")
df.show()

As you can see, nothing printed, this is because you only described what you want to do but you didn't really execute the dag. To run it:

In [ ]:
dag.run()

If you want to make it run automatically, use `with` statement

In [ ]:
with FugueWorkflow() as dag:
    df = dag.df([[0]],"a:int")
    df.show()

However, where did I setup the ExecutionEngine? So actually by default, `FugueWorkflow` will use `NativeExecutionEngine`, in order to setup your own ExecutionEngine, you can do:

In [ ]:
from pyspark.sql import SparkSession
from fugue_spark.execution_engine import SparkExecutionEngine


dag = FugueWorkflow()
df = dag.df([[0]],"a:int")
df.show()

# see all the expensive initialization is after the dag is constructed, the steps above are fast
# if you have any obvious issue, they will fail fast, 
# they are compile time problems and the below is all about runtime
spark_session = (SparkSession
                 .builder
                 .config("spark.executor.cores",4)
                 .getOrCreate())

my_engine = SparkExecutionEngine(spark_session)
dag.run(my_engine)

# You can also do the following, but you move the dag construction to be after building spark session
# so at least you have to wait until the spark session is initialized, you can know if there is compile problems
with FugueWorkflow(my_engine) as dag:
    df = dag.df([[0]],"a:int")
    df.show()


## Create Dataframes

In [ ]:
from fugue.workflow import FugueWorkflow
import pandas as pd
from fugue.dataframe import ArrayDataFrame

dag = FugueWorkflow()
# You can pass in raw data, pandas dataframe or Fugue DataFrames
dag.df([[0]],"a:int").show()
dag.df(pd.DataFrame([[0]], columns=["a"])).show()
dag.df(ArrayDataFrame([[0]],"a:int")).show()
dag.run()

## Create DataFrame with Creator

Please read [creator tutorial](./creator.ipynb) for details how to write creators in different ways. Here is just one way.

In [ ]:
from fugue.workflow import FugueWorkflow
from typing import List, Any

#schema: a:int
def create_single(n=1) -> List[List[Any]]:
    return [[n]]

with FugueWorkflow() as dag:
    dag.create(create_single).show()
    dag.create(create_single, params={"n":2}).show()

## Process DataFrame with Processor

Please read [processor tutorial](./processor.ipynb) for details how to write processor in different ways

In [ ]:
from fugue.workflow import FugueWorkflow
from fugue.execution import ExecutionEngine
from fugue.dataframe import DataFrame, ArrayDataFrame
from typing import List, Any

def ct(e:ExecutionEngine, df1:DataFrame, df2:DataFrame) -> DataFrame:
    # if the function signature has ExecutionEngine, the creator becomes engine aware
    # here we need engine to persist the dataframes before counting
    c1, c2 = e.persist(df1).count(), e.persist(df2).count()
    return ArrayDataFrame([[c1,c2]],"c1:long,c2:long")

with FugueWorkflow() as dag:
    a=dag.df([[0]],"a:int")
    b=dag.df([[1],[3]],"a:int")
    dag.process(a,b,using=ct).show()

## Output DataFrame with Outputter

Please read [outputter tutorial](./outputter.ipynb) for details how to write outputter in different ways

In [ ]:
from fugue.workflow import FugueWorkflow
from fugue.dataframe import DataFrame, ArrayDataFrame, DataFrames
from typing import List, Any

def print_array(dfs:DataFrames) -> None:
    for df in dfs.values():
        print(df.as_array())
    print("---")

with FugueWorkflow() as dag:
    a=dag.df([[0]],"a:int")
    b=dag.df([[1],[3]],"a:int")
    dag.output(a,b,using=print_array)
    # you can also directly call output from a dag dataframe
    a.output(print_array)

## Transform & CoTransform

Please read [transformer tutorial](./transformer.ipynb) and [cotransformer tutorial](./cotransformer.ipynb) for details how to write transformers in different ways

In this example we want to get nth smallest or largest row of each partition of column `a`

In [ ]:
from fugue.workflow import FugueWorkflow
from fugue.dataframe import DataFrame, ArrayDataFrame, DataFrames
from fugue_spark.execution_engine import SparkExecutionEngine
import pandas as pd
import numpy as np
from timeit import timeit
from typing import Iterable, List, Any

def helper(ct=20000) -> pd.DataFrame:
    return pd.DataFrame(np.random.randint(0,100,size=(ct, 3)), columns=list('abc'))

# schema: *
def nth(df:Iterable[List[Any]], n) -> Iterable[List[Any]]:
    for row in df:
        if n==0:
            yield row
            return
        n-=1
        
dag = FugueWorkflow()
df = dag.create(helper).persist() # add persist here so the data will not be regenerated
largest_4 = df.partition(by=["a"],presort="b DESC, c DESC").transform(nth, params={"n":4})
smallest_4 = df.partition(by=["a"],presort="b,c").transform(nth, params={"n":4})
largest_4.persist().show()
smallest_4.persist().show()

print(timeit(lambda: dag.run(), number=1)) # using native python

engine = SparkExecutionEngine()
print(timeit(lambda: dag.run(engine), number=1)) # if engine is well configured, this logic can handle very large dataset

As you can see, on such small data, Spark local mode is way slower than running on `NativeExecutionEngine`. In practice, when we unit test Fugue code, try to use `NativeExecutionEngine` since their behavior should be consistent. This is guaranteed on Fugue level, you only choose the faster one in a certain scenario.

In [ ]:
from fugue.workflow import FugueWorkflow
from fugue.dataframe import DataFrame, ArrayDataFrame, DataFrames
from fugue_spark.execution_engine import SparkExecutionEngine
import pandas as pd
import numpy as np
from timeit import timeit
from typing import Iterable, List, Any

def helper(ct=20) -> pd.DataFrame:
    np.random.seed(0)
    return pd.DataFrame(np.random.randint(0,4,size=(ct, 3)), columns=list('abc'))

# schema: value:[str]
def to_str(df1:List[List[Any]], df2:List[List[Any]]) -> List[List[Any]]:
    return [[[df1.__repr__(),df2.__repr__()]]]
        
dag = FugueWorkflow()
df1 = dag.create(helper)
df2 = dag.create(helper)
df1.zip(df2,how="inner",partition={"by":["a"],"presort":"b,c"}).transform(to_str).show()

print(timeit(lambda: dag.run(), number=1))

engine = SparkExecutionEngine()
print(timeit(lambda: dag.run(engine), number=1))